# King County House Sales Analysis

**Author:** Lili Beit
***

## Overview

A large real estate firm in the Seattle area is seeking to maximize prices for home sellers.  My task is to use data from previous home sales to predict future prices.  The firm aims to cast a wide net and attract clients at all price points from throughout the county.

## Business Problem
The real estate firm operates throughout King County, which includes the metropolis of Seattle, as well as suburban and rural areas.  Home prices vary greatly between these diverse landscapes, as well as between neighborhoods in Seattle.  The firm needs to accurately price a home based on data such as its size, location, and number of bedrooms, in order to get the best sale price for its clients.  It needs a model that can generate a good estimate of value for homes in every part of the county.

## Data Understanding

To build a model to predict  prices, I used data from the King County House Sales dataset, which can be found here:

(https://www.kaggle.com/harlfoxem/housesalesprediction)

This dataset contains information on over 21,000 houses sold in King County between May, 2014 and May, 2015.  Although the median sale price is \\$450,000, the dataset also includes multi-million dollar homes.  At the top of the market are about 1,000 properties which sold between \\$1.2 million and \\$7.7 million, so the price data are right-skewed with a few very high outliers.

In addition to sale price, the dataset includes details about the homes, including square footage, lot square footage, number of bedrooms, zip code, and the dates when the houses were built, renovated, and sold.  Although the data seem mostly accurate, some values are missing, and many columns have outliers.

Definitions of all column names are below:

### Column Names and Descriptions for King County Data Set

* **id** - unique identifier for a house
* **date** - house was sold
* **price** -  is prediction target
* **bedrooms** -  number of bedrooms
* **bathrooms** -  number of bathrooms
* **sqft_living** -  footage of the home
* **sqft_lot** -  footage of the lot
* **floors** -  floors (levels) in house
* **waterfront** - House which has a view to a waterfront
* **view** - Has been viewed
* **condition** - How good the condition is ( Overall )
* **grade** - overall grade given to the housing unit, based on King County grading system
* **sqft_above** - square footage of house apart from basement
* **sqft_basement** - square footage of the basement
* **yr_built** - Built Year
* **yr_renovated** - Year when house was renovated
* **zipcode** - zip
* **lat** - Latitude coordinate
* **long** - Longitude coordinate
* **sqft_living15** - The square footage of interior housing living space for the nearest 15 neighbors
* **sqft_lot15** - The square footage of the land lots of the nearest 15 neighbors


## Import Data and Split into Training and Test Sets

In [2]:
# import packages

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

import numpy as np

from itertools import combinations

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

import statsmodels.api as sm
from statsmodels.formula.api import ols

import eli5

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
import seaborn as sns

In [ ]:
# import data

data = pd.read_csv('data/kc_house_data.csv')

In [ ]:
# split data into test and training sets

# choose relevant columns:

X=data.drop(columns=['price'])

y=data['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(len(X_train), len(X_test), len(y_train), len(y_test))

In [ ]:
# concatenate X_train and y_train back together for initial exploration

train_data = pd.concat([X_train, y_train], axis=1)
train_data

## Data Cleaning and Preprocessing

### Right-Skewed Columns and Outliers

While exploring the data, I found several columns that are right skewed, including:
* price
* bedrooms
* bathrooms
* sqft_living
* sqft_lot
* sqft_above
* sqft_living15
* sqft_lot15

In this section, I investigated the right-skewed columns.  I found that although the data do not appear inaccurate, many columns have high outliers.  I removed the highest outliers in price, square footage, and lot square footage from the data to improve the model's accuracy for the remaining homes.  Later, I will use log transformation on some columns to reduce the effect of the skewness.

In [ ]:
# explore training data

train_data.head(500)
train_data.info()
train_data.describe()

# questions and observations:
# high outliers in price, bedrooms, bathrooms, sqft_living, sqft_lot
# null values in waterfront, view, yr_renovated
# waterfront has lots of 0s in addition to null values
# need to turn date (sale date) into a date
# need to turn sqft_basement into a float (but it has some non-number values, like ?)

In [ ]:
# check data set time frame

pd.to_datetime(train_data['date']).describe()

# homes sold between May 2014 and May 2015

In [ ]:
# investigate price outliers
plt.boxplot(train_data['price'])
plt.xlabel('homes')
plt.ylabel('price'); # looks like outliers are probably accurate, but may decrease the model's efficacy

In [ ]:
# find 95th percentile of price data
train_data['price'].quantile(0.95)

In [ ]:
# look at highest price outliers

data_price_outliers = train_data.loc[data.price >= 1170000].sort_values(by='price', ascending=False)
data_price_outliers.head(500)
data_price_outliers.describe()

In [ ]:
train_data['price'].hist(bins = 10)
plt.xticks(rotation = 'vertical')
plt.xlabel('price')
plt.ylabel('number of homes');

In both the box plot and the histogram above, prices look extremely unusual above $3 million.  There are 36 homes at or above this sale price in the training data.  I will remove these to improve the model later.

In [ ]:
# how many homes sold at or above $3 million?

train_data['price'].loc[train_data['price'] >= 3000000].count() #36 homes

In [ ]:
# drop rows with price > $3 million for training data

train_data = train_data.loc[train_data['price'] < 3000000]
len(train_data)

In [ ]:
# make the same change to the test data

test_data = pd.concat([X_test, y_test], axis=1)
test_data['price'].loc[test_data['price'] >= 3000000].count() #15 homes
test_data = test_data.loc[test_data['price'] < 3000000]
len(test_data)

In [ ]:
# looking at row detail, max values for bedrooms, bathrooms, sqft_living & sqft_above seem plausible
# but for sqft_living and sqft_above, there is one home with a huge outlier
# same with sqft_lot

# outliers = train_data.sort_values(by='bedrooms', ascending=False).head(500)
# outliers = train_data.sort_values(by='bathrooms', ascending=False).head(500)
# outliers = train_data.sort_values(by='sqft_living', ascending=False).head(500)
# outliers = train_data.sort_values(by='sqft_above', ascending=False).head(500)
outliers = train_data.sort_values(by='sqft_lot', ascending=False).head(500)
outliers

In [ ]:
# investigate sqft_living outliers

plt.boxplot(train_data.sqft_living)
plt.xlabel('homes')
plt.ylabel('sqft_living');

# the max value may be accurate, but let's drop it to improve model accuracy

In [ ]:
# drop sqft_living > 8000 from training and testing data

train_data = train_data.loc[train_data['sqft_living'] <= 8000]
test_data = test_data.loc[test_data['sqft_living'] <= 8000]

In [ ]:
# investigate sqft_lot outliers

plt.boxplot(train_data.sqft_lot)
plt.xlabel('homes')
plt.ylabel('sqft_lot');

# the max values may be accurate, but let's drop them to improve model accuracy

In [ ]:
# remove sqft_lot > 600,000 from train and test data

train_data = train_data.loc[train_data['sqft_lot'] <= 600000]
test_data = test_data.loc[test_data['sqft_lot'] <= 600000]

### Null values

Four columns had null values: 'waterfront', 'yr_renovated', 'sqft_basement', and 'view'.  Since fewer than 1% of properties were marked as having waterfront views, I dropped this column from the analysis.  I replaced 'yr_renovated' with a binary column showing whether or not the home was marked renovated in any year.  I replaced the null values in 'sqft_basement' (which appeared as ? in the data) with zeros, since the median of the non-null values in this column was also zero.  Since 'view' refers to the number of times a house had been viewed (not whether it has a nice view), I dropped this column from the analysis.

In [ ]:
# investigate null values in waterfront

train_data.waterfront.value_counts() # binary - 1 or 0
train_data.waterfront.isna().sum() #1647 null values out of 21596
train_data.waterfront.value_counts() 

# Only 89 homes are marked as waterfront -- less than 1% of data
# So I'll drop this feature from the analysis

In [ ]:
# deal with null values in yr_renovated

train_data['yr_renovated'].value_counts().head(50)
# 11869 values are 0 (meaning no true value)

nulls = train_data['yr_renovated'].isna().sum()
nulls #2692 values are null

# many of the values with years are old, e.g. 1950's-1990's

In [ ]:
# create a new column showing homes renovated or not
train_data['renovated'] = np.where(train_data['yr_renovated'] > 0, 1, 0)
train_data['renovated'].value_counts() # only 511 homes show a year renovated

# make same change to testing data
test_data['renovated'] = np.where(test_data['yr_renovated'] > 0, 1, 0)

In [ ]:
# deal with non-number values in sqft_basement
train_data['sqft_basement'].value_counts() # continuous variable, but has many '?' values
# also many 0 values.  Not sure if these homes truly do not have basements.

# per cent of data that is missing:
missing_sqft_basement = round((len(train_data.loc[train_data['sqft_basement'] == '?'])/len(train_data))*100, 2)
print(missing_sqft_basement, "% of basement data is missing")

# per cent of data that is zero:
missing_sqft_basement = round((len(train_data.loc[train_data['sqft_basement'] == '0.0'])/len(train_data))*100, 2)
print(missing_sqft_basement, "% of basement data is zero")

In [ ]:
# for now, let's fill missing values with zero, since that's the median

# replace all '?' values with '0'
train_data.loc[train_data['sqft_basement'] == '?', 'sqft_basement'] = '0'

# same for test data
test_data.loc[test_data['sqft_basement'] == '?', 'sqft_basement'] = '0'

In [ ]:
# now convert sqft_basement values to integers

train_data['sqft_basement'] = pd.to_numeric(train_data['sqft_basement'])

test_data['sqft_basement'] = pd.to_numeric(test_data['sqft_basement'])

train_data.info()

### Exploring Correlations

The analysis below shows that the strongest correlations with price (the target variable) are sqft_living, sqft_above, sqft_living15, grade, and bathrooms.
The strongest correlations between X variables are among these same columns -- all five are correlated with each other.  This multicolinearity could negatively impact a prediction model, so I will experiment with dropping combinations of multicolinear columns later on.
Of all the variables, only grade and bedrooms look normally distributed.  Most are right-skewed, including price.  Later, I will see if log transformations on these variables improve the model.

In [ ]:
# explore training data
# let's try a pairplot to see if anything stands out

cols_of_interest = [ 
                    'bedrooms', 
                    'bathrooms', 
                    'sqft_living',
                    'sqft_lot', 
                    'condition', 
                    'grade', 
                    'sqft_above',
                    'yr_built',  
                    'sqft_living15', 
                    'sqft_lot15',
                    'price']

sns.pairplot(train_data[cols_of_interest]);

# the strongest correlations with price are sqft_living, sqft_above, sqft_living15, grade, and bathrooms
# strongest correlations between X variables are among these same columns
# only grade and bedrooms look normally distributed

In [ ]:
# let's make a heatmap to be sure

sns.heatmap(train_data[cols_of_interest].corr())

# yes, confirms the observations above

In [ ]:
# let's look at the numbers

train_data[cols_of_interest].corr()

# sqft_living is the best predictor of price so far

In [ ]:
# let's look just at the correlations with price

train_data[cols_of_interest].corr()['price'].sort_values(ascending=False)

# interesting, sqft_living and grade are far above the rest
# grade is probably based in part on sqft_living

### Column Exclusion
Based on the analyses above, I decided to exclude the following columns from the model.  Justifications are provided below:

* id - the randomly assigned house id
* date - date sold, all are between May 2014 and May 2015.  May investigate impact of month later
* waterfront - less than 1% of homes marked as waterfront
* view - number of times the home has been viewed - not relevant for pricing homes newly on the market
* yr_renovated - missing values.  Turned into binary column 'renovated'
* 'lat' and 'long' - latitude and longitude of house - easier to pull location info with zipcode

## Modeling

In this section, I tested nine models, and selected Model 8 as the most effective.  \
\
I first calculated a model-less baseline with an R-squared of 0 and a Mean Absolute Error of \\$227K.  I then tested a baseline linear regression model without transforming any features of the data.  This produced an R-squared of 0.64 and 0.62 for the training and test sets respectively, and Mean Absolute Errors of \\$135K and \\$136K respectively. \
\
After experimenting with log-transforming the X variables and the target variable price, I was able to improve the metrics by log-transforming price, sqft_living15, and sqft_lot15.  By assigning zip codes to price-based classifications, I improved the R-squared to 0.83 for both the training and test data, with Mean Absolute Errors of \\$88K and \\$87K respectively. \
\
I also tested other strategies, such as reducing multicolinearity by dropping columns, omitting features with high p-values, and assigning the yr_built data to categories.  None of these changes improved the model.  However, omitting features with high p-values did not reduce the model's effectiveness either, so I kept this change in order to simplify the model.

In [ ]:
# split the preprocessed training and test sets back into X and y

# choose relevant columns:

X_train=train_data[['bedrooms', 
       'bathrooms', 
       'sqft_living', 
       'sqft_lot', 
       'floors', 
       'condition', 
       'grade',
       'sqft_above', 
       'sqft_basement', 
       'yr_built', 
       'zipcode',
       'sqft_living15', 
       'sqft_lot15', 
       'renovated']]

y_train=train_data['price']

X_test=test_data[['bedrooms', 
       'bathrooms', 
       'sqft_living', 
       'sqft_lot', 
       'floors', 
       'condition', 
       'grade',
       'sqft_above', 
       'sqft_basement', 
       'yr_built', 
       'zipcode',
       'sqft_living15', 
       'sqft_lot15', 
       'renovated']]

y_test=test_data['price']

print(len(X_train), len(X_test), len(y_train), len(y_test))

### Model 1: Model-less Baseline

In [ ]:
# for our first model-less baseline, let's use the mean price
# start with training set

mean_price = y_train.mean()
y_pred_train = np.full(shape=(len(X_train), 1), fill_value=mean_price)

# check r2
r2_baseline_train = round(r2_score(y_true=y_train, y_pred=y_pred_train), 6)

# check Mean Absolute Error
mae_baseline_train = round(mean_absolute_error(y_true=y_train, y_pred=y_pred_train), 2)

# check Root Mean Squared Error
rmse_baseline_train = round(np.sqrt(mean_squared_error(y_true=y_train, y_pred=y_pred_train)), 2)

print('Training Data', '\n', 
      'Mean Price:', round(mean_price, 2), '\n', 
      'R-Squared:', r2_baseline_train, '\n',
      'Mean Absolute Error:', mae_baseline_train, '\n',
      'Root Mean Squared Error:', rmse_baseline_train)

In [ ]:
# now let's calculate baseline r2, MAE, and RMSE for the test set

y_pred_test = np.full(shape=(len(X_test), 1), fill_value=mean_price)

r2_baseline_test = round(r2_score(y_true=y_test, y_pred=y_pred_test), 6)
mae_baseline_test = round(mean_absolute_error(y_true=y_test, y_pred=y_pred_test), 2)
rmse_baseline_test = round(np.sqrt(mean_squared_error(y_true=y_test, y_pred=y_pred_test)), 2)

print('Testing Data', '\n', 
      'Mean Price:', round(mean_price, 2), '\n', 
      'R-Squared:', r2_baseline_test, '\n',
      'Mean Absolute Error:', mae_baseline_test, '\n',
      'Root Mean Squared Error:', rmse_baseline_test)

In [ ]:
# create a function for evaluating models:

def evaluate_model(y_train, y_train_pred, y_test, y_test_pred):
    
    """Calculate evaluation metrics for the model: R-Squared, Mean Absolute Error, and Root Mean Squared Error
    
    Parameters
    ----------
    y_train: Series of true values from the training set target variable
    y_train_pred: Series of target variable values predicted by the model for the training set
    y_test: Series of true values from the test set target variable
    y_test_pred: Series of target variable values predicted by the model for the test set

    Returns
    -------
    Print of metrics for training and test sets"""

    # check train r2
    r2_train = round(r2_score(y_true=y_train, y_pred=y_train_pred), 6)

    # check train Mean Absolute Error
    mae_train = round(mean_absolute_error(y_true=y_train, y_pred=y_train_pred), 2)

    # check train Root Mean Squared Error
    rmse_train = round(np.sqrt(mean_squared_error(y_true=y_train, y_pred=y_train_pred)), 2)

    print('Training Data', '\n', 
          'R-Squared:', r2_train, '\n',
          'Mean Absolute Error:', mae_train, '\n',
          'Root Mean Squared Error:', rmse_train, '\n')
    
    # check test r2
    r2_test = round(r2_score(y_true=y_test, y_pred=y_test_pred), 6)

    # check test Mean Absolute Error
    mae_test = round(mean_absolute_error(y_true=y_test, y_pred=y_test_pred), 2)

    # check train Root Mean Squared Error
    rmse_test = round(np.sqrt(mean_squared_error(y_true=y_test, y_pred=y_test_pred)), 2)

    print('Testing Data', '\n', 
          'R-Squared:', r2_test, '\n',
          'Mean Absolute Error:', mae_test, '\n',
          'Root Mean Squared Error:', rmse_test)


In [ ]:
# the mean is not a good predictor of price!

# let's fit a baseline regression model

### Model 2: Baseline Linear Regression

In [ ]:
# first, let's scale the data so we can evaluate the coefficients of the baseline model

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# now do a linear regression

linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

y_train_pred2 = linreg.predict(X_train_scaled)
y_test_pred2 = linreg.predict(X_test_scaled)

evaluate_model(y_train, y_train_pred2, y_test, y_test_pred2)

In [ ]:
# that's better, but the model still only explains about 60% of the variance
# store as 'best_r2' for comparison

best_r2 = {'train': 0.635164, 'test': 0.618171}

In [ ]:
# let's plot training set residuals

fig, ax = plt.subplots(nrows=1, ncols=2, figsize = (18,6))

residuals_train = y_train-y_train_pred2
ax1 = plt.subplot(121)
ax1.scatter(y_train_pred2, residuals_train)
ax1.plot(y_train_pred2, [0 for i in range(len(y_train_pred2))])
plt.title('Training Data')
plt.xlabel('price predictions')
plt.ylabel('error')

residuals_test = y_test-y_test_pred2
ax2 = plt.subplot(122)
ax2.scatter(y_test_pred2, residuals_test)
ax2.plot(y_test_pred2, [0 for i in range(len(y_test_pred2))])
plt.title('Test Data')
plt.xlabel('price predictions')
plt.ylabel('error');

# cone-shaped residuals indicate heteroskedasticity
# means that as price increases, error increases as well
# will try log transformations to reduce the effect of outliers

In [ ]:
# run it in Statsmodels to check coefficients

model = sm.OLS(y_train, sm.add_constant(pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)))
results = model.fit()

results.summary()

# same R-squared as above
# sqft_lot and sqft_above have p-values above 0.05.  Experiment with removing these later

### Model 3: 7 Log-Transformed X Variables

In [ ]:
# let's look at the numeric variables.  Are they normally distributed?

numeric = ['bedrooms', 
       'bathrooms', 
       'sqft_living', 
       'sqft_lot', 
       'floors', 
       'condition', 
       'grade',
       'sqft_above', 
       'sqft_basement', 
       'yr_built',
       'sqft_living15', 
       'sqft_lot15'
             ]

num_cols = 3
if len(numeric)%num_cols == 0:
    num_rows = len(numeric)//num_cols
else:
    num_rows = (len(numeric)//num_cols)+1


fig, axs = plt.subplots(figsize=(12,20), nrows=num_rows, ncols=num_cols)


for feat in numeric:
    axs[numeric.index(feat)//num_cols, numeric.index(feat)%num_cols].hist(X_train[feat], bins=20)
    axs[numeric.index(feat)//num_cols, numeric.index(feat)%num_cols].set_title(feat)

In [ ]:
# numeric variables are not normally distributed
# let's try to log these and see if they become more normal
# don't include features with zeros, like sqft_basement

non_zero = ['bedrooms', 
       'bathrooms', 
       'sqft_living', 
       'sqft_lot', 
       'floors', 
       'condition', 
       'grade',
       'sqft_above', 
       'yr_built',
       'sqft_living15', 
       'sqft_lot15'
             ]

X_train_logged = X_train.copy()

for feat in non_zero:
    X_train_logged[feat] = X_train_logged[feat].map(lambda x: np.log(x))


In [ ]:
# Did it help?  Make more histograms

num_cols = 3
if len(non_zero)%num_cols == 0:
    num_rows = len(non_zero)//num_cols
else:
    num_rows = (len(non_zero)//num_cols)+1


fig, axs = plt.subplots(figsize=(12,20), nrows=num_rows, ncols=num_cols)


for feat in non_zero:
    axs[non_zero.index(feat)//num_cols, non_zero.index(feat)%num_cols].hist(X_train_logged[feat], bins=20)
    axs[non_zero.index(feat)//num_cols, non_zero.index(feat)%num_cols].set_title(feat)

In [ ]:
# it helped - some variables look more normally distributed
# like sqft_living, sqft_lot, grade, sqft_above, sqft_living15, sqft_lot15
# and to a lesser extent, bedrooms and grade too

In [ ]:
# what if we build a model with just the above columns logged

# build a new X_train with just the above features logged

to_log = ['bedrooms', 
       'sqft_living', 
       'sqft_lot', 
       'grade',
       'sqft_above', 
       'sqft_living15', 
       'sqft_lot15'
             ]

X_train3 = X_train.copy()

for feat in to_log:
    X_train3[feat] = X_train3[feat].map(lambda x: np.log(x))
    
# log the test data

X_test3 = X_test.copy()

for feat in to_log:
    X_test3[feat] = X_test3[feat].map(lambda x: np.log(x))

In [ ]:
# build a function to scale the X variables, and do a linear regression

def scale_lin_reg(X_train, y_train, X_test):
    
    """Perform standard scaling and linear regression given training set and test set X-variables
    
    Parameters
    ----------
    X_train: DataFrame of training set input variables
    y_train: Array of true values from the training set target variable
    X_test: DataFrame of test set input variables

    Returns
    -------
    y_train_pred: Series of training set target variable predictions
    y_test_pred: Series of test set target variable predictions 
    """
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    linreg = LinearRegression()
    linreg.fit(X_train_scaled, y_train)

    y_train_pred = linreg.predict(X_train_scaled)
    y_test_pred = linreg.predict(X_test_scaled)
        
    return(y_train_pred, y_test_pred)


In [ ]:
# let's scale and do a linear regression on the transformed data, to return y_train_pred and y_test_pred
# the inputs X_train3 and X_test3 have 7 features logged

y_train_pred3, y_test_pred3 = scale_lin_reg(X_train=X_train3, y_train=y_train, X_test=X_test3)

In [ ]:
# now let's evaluate that model
evaluate_model(y_train=y_train, y_train_pred=y_train_pred3, y_test=y_test, y_test_pred=y_test_pred3)

In [ ]:
best_r2

# oh no!  It didn't help!  R2 got worse

### Model 4: 2 Logged X Variables

In [ ]:
# these two features have the most improvement in normality after log transformations:
# sqft_living15
# sqft_lot15

# what if we just log these?

to_log = [
       'sqft_living15', 
       'sqft_lot15'
             ]

X_train4 = X_train.copy()

for feat in to_log:
    X_train4[feat] = X_train4[feat].map(lambda x: np.log(x))

# log the test data

X_test4 = X_test.copy()

for feat in to_log:
    X_test4[feat] = X_test4[feat].map(lambda x: np.log(x))

In [ ]:
# let's scale and do a linear regression on the transformed data, to return y_train_pred and y_test_pred
# the inputs X_train4 and X_test4 have 2 features logged

y_train_pred4, y_test_pred4 = scale_lin_reg(X_train=X_train4, y_train=y_train, X_test=X_test4)

In [ ]:
# evaluate the model

evaluate_model(y_train=y_train, y_train_pred=y_train_pred4, y_test=y_test, y_test_pred=y_test_pred4)

In [ ]:
best_r2

# That is a very slight improvement over the baseline model
# Price (target variable) was also right-skewed.  Let's try logging this as well.

### Model 5: Two Logged X Variables and Logged Target Variable

In [ ]:
y_train_logged = y_train.copy()
y_train_logged = y_train_logged.map(lambda y: np.log1p(y))

fig, ax = plt.subplots(figsize = (15,4), nrows=1, ncols=2)

ax[0].hist(y_train, bins=20)
ax[0].set_title('y_train')

ax[1].hist(y_train_logged, bins=20)
ax[1].set_title('y_train_logged');

# the logged y_train definitely looks more normally distributed

In [ ]:
# log y_test as well

y_test_logged = y_test.copy()
y_test_logged = y_test_logged.map(lambda y: np.log1p(y))

fig, ax = plt.subplots(figsize = (15,4), nrows=1, ncols=2)

ax[0].hist(y_test, bins=20)
ax[0].set_title('y_test')

ax[1].hist(y_test_logged, bins=20)
ax[1].set_title('y_test_logged');

# the logged y_test looks more normally distributed too

In [ ]:
# now let's run and evaluate the model with X_train4, X_test4, y_train_logged and y_test_logged

y_train_pred5, y_test_pred5 = scale_lin_reg(X_train=X_train4, y_train=y_train_logged, X_test=X_test4)

In [ ]:
# create a function to only evaluate R-squared, since MAE and RMSE must use unlogged price predictions

def eval_r2(y_train, y_train_pred, y_test, y_test_pred):
    """Evalute R-Squared for training and test predictions
    
    Parameters
    ----------
    y_train: Array of true values from the training set target variable
    y_train_pred: Array of target variable values predicted by the model for the training set
    y_test: Array of true values from the test set target variable
    y_test_pred: Array of target variable values predicted by the model for the test set

    Returns
    -------
    Print of R-Squared for training and test sets"""

    # calculate r2 using logged target variable
    r2_train = round(r2_score(y_true=y_train, y_pred=y_train_pred), 6)
    r2_test = round(r2_score(y_true=y_test, y_pred=y_test_pred), 6)

    print('Training Data', '\n', 
          'R-Squared:', r2_train, '\n')
    
    print('Test Data', '\n', 
          'R-Squared:', r2_test)
    

In [ ]:
eval_r2(y_train=y_train_logged, y_train_pred=y_train_pred5, y_test=y_test_logged, y_test_pred=y_test_pred5)

In [ ]:
# great, it helped a little, but need to unlog y_train_pred5 and y_test_pred5 to measure price errors

# create a function to unlog predictions and measure MAE and RMSE
def unlog_MAE_RMSE(y_train, y_train_logged_pred, y_test, y_test_logged_pred):
    
    """Unlog target variable values, and evaluate Mean Absolute Error and Root Mean Squared Error for training and test predictions
    
    Parameters
    ----------
    y_train: Series of true values from the training set target variable
    y_train_logged_pred: Series of target variable values predicted using a logged target variable; training set
    y_test: Series of true values from the test set target variable
    y_test_pred: Series of target variable values predicted using a logged target variable; test set

    Returns
    -------
    Print of Mean Absolute Error and Root Mean Squared Error for training and test sets"""
    
    # unlog target variable predictions to measure MAE and RMSE
    y_train_pred_exp = np.expm1(y_train_logged_pred)
    y_test_pred_exp = np.expm1(y_test_logged_pred)
    
    # check Mean Absolute Error
    mae_train = round(mean_absolute_error(y_true=y_train, y_pred=y_train_pred_exp), 2)
    mae_test = round(mean_absolute_error(y_true=y_test, y_pred=y_test_pred_exp), 2)

    # check Root Mean Squared Error
    rmse_train = round(np.sqrt(mean_squared_error(y_true=y_train, y_pred=y_train_pred_exp)), 2)
    rmse_test = round(np.sqrt(mean_squared_error(y_true=y_test, y_pred=y_test_pred_exp)), 2)

    print('Training Data', '\n', 
          'Mean Absolute Error:', mae_train, '\n',
          'Root Mean Squared Error:', rmse_train, '\n')
    
    print('Test Data', '\n', 
          'Mean Absolute Error:', mae_test, '\n',
          'Root Mean Squared Error:', rmse_test, '\n')

In [ ]:
unlog_MAE_RMSE(y_train=y_train, y_train_logged_pred=y_train_pred5, y_test=y_test, y_test_logged_pred=y_test_pred5)

In [ ]:
# update best_r2

best_r2['train'] = 0.656332
best_r2['test'] = 0.62867

### Model 6: Two Logged X Variables, Logged Target Variable, and Zip Code Categories

In [ ]:
# let's try to assign zip codes to price categories

X_train6 = X_train4.copy() # use X_train4, which had two features logged

X_train6.info()

In [ ]:
X_train6['zipcode'].value_counts().count() # 70 different zips
X_train6['zipcode'].value_counts()

In [ ]:
zips=pd.concat([X_train6['zipcode'], pd.DataFrame(y_train)['price']], axis=1)
zips

In [ ]:
# find mean price by zip to see if any stand out

zips_pivot = zips.pivot_table(values='price', index='zipcode', ).sort_values(by='price', ascending=False)
zips_pivot.plot(kind='bar', figsize=(16,10))
plt.ylabel('mean house price')
plt.legend;

plt.savefig('price_by_zip_code')

# yes, some do stand out!  the top four, the bottom three
# what if I classified them based on price? I can make a dictionary

In [ ]:
# create a dictionary of zip codes and classifications

ordered_zip_list = list(zips_pivot.index)
zip_dict = {}

# display all zips and index in price-ordered list for eyeballing

for i in ordered_zip_list:
    print(i, ordered_zip_list.index(i))

In [ ]:
# classify zips in dict:

zip_dict[ordered_zip_list[0]] = 'Zip Class 1'

# make a function to add entries more easily
def add_to_zip_dict(list_index_start, list_index_stop, category):
    
    """Add entries to zip_dict based on their index in ordered_zip_list.
    
    Parameters
    ----------
    list_index_start: index in ordered_zip_list of the first zip code to enter
    list_index_stop: index in ordered_zip_list of the zip code to stop at
    category: zip class to assign to these entries"""
    
    for i in ordered_zip_list[list_index_start:list_index_stop]:
        zip_dict[i] = category

add_to_zip_dict(1, 4, 'Zip Class 2')
add_to_zip_dict(4, 13, 'Zip Class 3')
add_to_zip_dict(13, 34, 'Zip Class 4')
add_to_zip_dict(34, 49, 'Zip Class 5')
add_to_zip_dict(49, 67, 'Zip Class 6')
add_to_zip_dict(67, 70, 'Zip Class 7')

zip_dict

In [ ]:
# add classification column to training data

X_train6['zip_class'] = X_train6['zipcode'].map(lambda x: zip_dict[x])
X_train6

In [ ]:
# one hot encode classification column and drop zipcode and zip_class columns

zip_class_columns = pd.get_dummies(X_train6['zip_class'], drop_first=True)
zip_class_columns

X_train6 = pd.concat([X_train6, zip_class_columns], axis=1)
X_train6.drop(columns=['zipcode','zip_class'], inplace=True)

In [ ]:
# add same features to test set

X_test6 = X_test4
X_test6['zip_class'] = X_test6['zipcode'].map(lambda x: zip_dict[x])

zip_class_columns = pd.get_dummies(X_test6['zip_class'], drop_first=True)
X_test6 = pd.concat([X_test6, zip_class_columns], axis=1)
X_test6.drop(columns=['zipcode','zip_class'], inplace=True)

In [ ]:
X_train6 #looks good
X_test6 #looks good

In [ ]:
# generate predictions

y_train_pred6, y_test_pred6 = scale_lin_reg(X_train=X_train6, y_train=y_train_logged, X_test=X_test6)

In [ ]:
# evaluate model using R-squared

eval_r2(y_train=y_train_logged, y_train_pred=y_train_pred6, y_test=y_test_logged, y_test_pred=y_test_pred6)

In [ ]:
# to evaluate MAE and RMSE, unlog y_train_pred6 and y_test_pred6

unlog_MAE_RMSE(y_train=y_train, y_train_logged_pred=y_train_pred6, y_test=y_test, y_test_logged_pred=y_test_pred6)

In [ ]:
# great, this really helped!
# update best_r2

best_r2 = {'train': 0.831058, 'test': 0.825734}

In [ ]:
# let's look at the training set residuals:

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15,6))

residuals_train6 = y_train_logged-y_train_pred6
ax1 = plt.subplot(121)
plt.scatter(y_train_pred6, residuals_train6)
plt.plot(y_train_pred6, [0 for i in range(len(y_train_pred6))])
plt.title('Training Data')
plt.xlabel('logged price predictions')
plt.ylabel('error')

residuals_test6 = y_test_logged-y_test_pred6
ax2 = plt.subplot(122)
plt.scatter(y_test_pred6, residuals_test6)
plt.plot(y_test_pred6, [0 for i in range(len(y_test_pred6))])
plt.title('Test Data')
plt.xlabel('logged price predictions')
plt.ylabel('error');

# looks better than the cone shape

In [ ]:
# look at coefficients

model = sm.OLS(y_train_logged, sm.add_constant(pd.DataFrame(X_train6, columns=X_train6.columns, index=X_train6.index)))
results = model.fit()

results.summary()

# sqft_above and sqft_basement have high p-values
# experiment with removing these later

In [ ]:
# which variables are most closely correlated with price?
X_train6.corrwith(y_train_logged).sort_values(ascending=False)

# grade and sqft_living, as in the original data

In [ ]:
# plot variables that are closely correlated with price, for presentation to non-technical stakeholders

plt.subplots(figsize=(15,12));

ax1 = plt.subplot(221)
sns.regplot(X_train6['sqft_living'], y_train_logged, line_kws={"color": "blue"})
plt.title('Training Data')
plt.ylabel('logged price')

ax2 = plt.subplot(222)
sns.regplot(X_test6['sqft_living'], y_test_logged, line_kws={"color": "blue"})
plt.title('Test Data')
plt.ylabel('logged price')

ax3 = plt.subplot(223)
sns.regplot(X_train6['sqft_living'], y_train, line_kws={"color": "blue"})
plt.title('Training Data')

ax4 = plt.subplot(224)
sns.regplot(X_test6['sqft_living'], y_test, line_kws={"color": "blue"})
plt.title('Test Data');

In [ ]:
plt.subplots(figsize=(15,12));

ax1 = plt.subplot(221)
sns.regplot(X_train6['grade'], y_train_logged, line_kws={"color": "blue"})
plt.title('Training Data')
plt.ylabel('logged price')

ax2 = plt.subplot(222)
sns.regplot(X_test6['grade'], y_test_logged, line_kws={"color": "blue"})
plt.title('Test Data')
plt.ylabel('logged price')

ax3 = plt.subplot(223)
sns.regplot(X_train6['grade'], y_train, line_kws={"color": "blue"})
plt.title('Training Data')

ax4 = plt.subplot(224)
sns.regplot(X_test6['grade'], y_test, line_kws={"color": "blue"})
plt.title('Test Data');

In [ ]:
plt.subplots(figsize=(15,12));

ax1 = plt.subplot(221)
sns.regplot(X_train6['sqft_living15'], y_train_logged, line_kws={"color": "blue"})
plt.title('Training Data')
plt.xlabel('logged square footage of 15 nearest neighbors')
plt.ylabel('logged price')

ax2 = plt.subplot(222)
sns.regplot(X_test6['sqft_living15'], y_test_logged, line_kws={"color": "blue"})
plt.title('Test Data')
plt.xlabel('logged square footage of 15 nearest neighbors')
plt.ylabel('logged price')

ax3 = plt.subplot(223)
sns.regplot(np.expm1(X_train6['sqft_living15']), y_train, line_kws={"color": "blue"})
plt.title('Training Data')
plt.xlabel('square footage of 15 nearest neighbors')

ax4 = plt.subplot(224)
sns.regplot(np.expm1(X_test6['sqft_living15']), y_test, line_kws={"color": "blue"})
plt.title('Test Data')
plt.xlabel('square footage of 15 nearest neighbors');

In [ ]:
# save figures for presentation

plt.subplots(figsize=(15,6));

ax1 = plt.subplot(121)
sns.regplot(X_train6['sqft_living'], y_train, line_kws={"color": "blue"})
plt.title('Square Footage vs Price', fontsize=20)
plt.xlabel('Living Space Square Footage', fontsize=16)
plt.ylabel('Home Price in Dollars', fontsize=16)

ax3 = plt.subplot(122)
sns.regplot(np.expm1(X_train6['sqft_living15']), y_train, line_kws={"color": "blue"})
plt.title('Square Footage of 15 Closest Neighbors vs Price', fontsize=20)
plt.xlabel('Square Footage of 15 Nearest Neighbors', fontsize=16)
plt.ylabel('Home Price in Dollars', fontsize=16)

plt.subplots_adjust(wspace = 0.3)

plt.savefig('images/regplots')

### Model 7: Testing Removing Multicolinear Columns

In [ ]:
# let's see if removing multicolinearity helps
# find top correlations
# code from Flatiron Data Science course's Multicollinearity Lab

df=X_train6.corr().abs().stack().reset_index().sort_values(0, ascending=False)

# zip the variable name columns
df['pairs'] = list(zip(df.level_0, df.level_1))

# set index to pairs
df.set_index(['pairs'], inplace = True)

#d rop level columns
df.drop(columns=['level_1', 'level_0'], inplace = True)

# rename correlation column as cc rather than 0
df.columns = ['cc']

# drop duplicates
df.drop_duplicates(inplace=True)

In [ ]:
df[(df.cc>.75) & (df.cc <1)]

# high correlations among these four variables:
# sqft_living, sqft_above, grade, bathrooms

In [ ]:
best_r2

In [ ]:
# iterate thru combinations of highly correlated variables to see if dropping them increases r2

correlated = ['sqft_living',
              'sqft_above',
              'grade',
              'bathrooms']

combs_list=[]

for n in range(1,4):
    
    comb = combinations(correlated,n)
    combs_list = combs_list + list(comb)
    
combs_list

for c in combs_list: 
    print(c)
    X_train7 = X_train6.drop(columns = list(c))
    X_test7 = X_test6.drop(columns = list(c))
    y_train_pred7, y_test_pred7 = scale_lin_reg(X_train=X_train7, y_train=y_train_logged, X_test=X_test7)
    eval_r2(y_train=y_train_logged, y_train_pred=y_train_pred7, y_test=y_test_logged, y_test_pred=y_test_pred7)
    print('\n')

# despite the multicolinearity, dropping combinations of these columns does not result in an improved R2
# dropping sqft_above returns almost exactly the same result

### Model 8: Test removing features with high p-values

In [ ]:
# according to the statsmodels output, p-values for sqft_above and sqft_basement were above 0.05

X_train8 = X_train6.copy()
X_train8.drop(columns=['sqft_above', 'sqft_basement'], inplace=True)

X_test8 = X_test6.copy()
X_test8.drop(columns=['sqft_above', 'sqft_basement'], inplace=True)


In [ ]:
y_train_pred8, y_test_pred8 = scale_lin_reg(X_train=X_train8, y_train=y_train_logged, X_test=X_test8)


In [ ]:
eval_r2(y_train=y_train_logged, y_train_pred=y_train_pred8, y_test=y_test_logged, y_test_pred=y_test_pred8)


In [ ]:
unlog_MAE_RMSE(y_train=y_train, y_train_logged_pred=y_train_pred8, y_test=y_test, y_test_logged_pred=y_test_pred8)


In [ ]:
best_r2
# no improvement in r2 for Model 8, but let's keep this model since at least it reduces multicolinearity
# and removes coefficients with high p-values

In [ ]:
# look at coefficients for Model 8

model = sm.OLS(y_train_logged, sm.add_constant(pd.DataFrame(X_train8, columns=X_train8.columns, index=X_train8.index)))
results = model.fit()

results.summary()

### Model 9: Experiment with categorizing year built

In [ ]:
# made a df to add price back to X variables table, so we can make a pivot table
built_df = pd.concat([X_train8, y_train], axis=1)
built_pivot = built_df.pivot_table(values='price', index='yr_built', ).sort_values(by='yr_built')

# plot a bar graph to look at possible categories
built_pivot.plot(kind='bar', figsize=(16,9))
plt.ylabel('mean house price')
plt.legend;

built_df

# hmmm, almost looks like older homes and new homes are highly valued, while homes in the middle are not

In [ ]:
# create a dictionary of years, showing the corresponding categories

years_list = list(built_pivot.index)
years_dict = {}

for i in years_list[0:41]:
    years_dict[i] = 'pre-war'
    
for i in years_list[41:88]:
    years_dict[i] = 'mid-century'

for i in years_list[88:117]:
    years_dict[i] = 'recent'
    
years_dict

In [ ]:
# add a column with yr_built categories

built_df['built_cat'] = built_df['yr_built'].map(lambda x: years_dict[x])
built_df

In [ ]:
# one hot encode classification column

built_cat_columns = pd.get_dummies(built_df['built_cat'], drop_first=True)
built_cat_columns

built_df = pd.concat([built_df, built_cat_columns], axis=1)
built_df.drop(columns=['yr_built','built_cat'], inplace=True)

In [ ]:
built_df

In [ ]:
# now we just have to stick these columns back onto the training and test sets

#training set first
X_train9 = X_train8.copy()

columns_to_add = built_df[['pre-war', 'recent']]
X_train9 = pd.concat([X_train9, columns_to_add], axis=1)
X_train9.drop(columns='yr_built', inplace=True)

In [ ]:
# now do test set

X_test9 = X_test8.copy()

X_test9['built_cat'] = X_test9['yr_built'].map(lambda x: years_dict[x])
built_cat_columns = pd.get_dummies(X_test9['built_cat'], drop_first=True)

X_test9 = pd.concat([X_test9, built_cat_columns], axis=1)
X_test9.drop(columns=['yr_built','built_cat'], inplace=True)

X_test9

In [ ]:
# let's test model 9!

y_train_pred9, y_test_pred9 = scale_lin_reg(X_train=X_train9, y_train=y_train_logged, X_test=X_test9)

In [ ]:
eval_r2(y_train=y_train_logged, y_train_pred=y_train_pred9, y_test=y_test_logged, y_test_pred=y_test_pred9)

# R-squared is less than for Model 8
# so, segmenting year_built into categories does not help explain any variance
# perhaps this variance can be explained by square footage and location alone

In [ ]:
best_r2

In [ ]:
model = sm.OLS(y_train_logged, sm.add_constant(pd.DataFrame(X_train9, columns=X_train9.columns, index=X_train9.index)))
results = model.fit()

results.summary()
# interesting, now floors has the highest p-value

## Conclusions and Future Work

To accurately price homes in King County, the real estate firm should use a model that segments zip codes into price-based categories, such as Model 8.  This model combines data about house features that are highly correlated with price, such as square footage, with knowledge of the mean house price of each zip code, to produce predictions that explain 83% of the variance from the mean price.  Model 8 is a significant improvement over the baseline regression model, which only explains 63% of the variance.  In addition, while the baseline regression model's predictions were an average of \\$136K off from the actual prices of the test data, Model 8's Mean Absolute Error was only $87K for the test data. \
\
Square footage and grade have the strongest positive correlation with price, but the model vastly improved after zip code classifications were included.  Unsurprisingly, location seems extremely important to home buyers in the Seattle area, which is a diverse landscape that includes, urban, suburban, and rural neighborhoods. \
\
Much work remains to investigate potential improvements to this model.  In particular, including interactions among X variables may increase the model's accuracy.  Since square footage and zip code are such powerful predictors of price, perhaps an interaction between these variables would enhance the model.  Also, since zip code classification was so effective in improving the model, perhaps including a few more zip classes would help by segmenting the market even further.  

In addition, the month when the house was sold may affect price, and was not tested in these models.  Also not tested was a feature that would indicate whether the house was recently renovated, for example in the past 20 years.  It may also help to programmatically iterate through the X-variables to select the best features for inclusion in the model.

Finally, a handful of properties (less than half of one per cent) must be excluded from this model.  Creating models that can generate predictions for these homes as well would benefit the real estate firm.